# 🚀 Generación de Dataset RAGAS con Qwen 2.5 14B en Google Colab (SIMPLIFICADO)

**Versión:** Sin Django - Solo 2 archivos necesarios  
**Objetivo:** Generar `ragas_evaluation_dataset.json` usando GPU de Colab (3-4x más rápido que local)

## ✅ Ventajas de esta versión:
- ✅ **Solo 2 archivos** necesarios (vs 4 en versión completa)
- ✅ **Sin Django** - configuración más simple
- ✅ **Misma calidad** - mismo modelo Qwen 2.5 14B y mismo RAG
- ✅ **Más rápido** - menos dependencias que instalar

## 📋 Requisitos:
1. Google Colab con GPU T4 (gratuita)
2. Archivo `backend_colab_simple.zip` con:
   - `rag_service.py` (tu sistema RAG completo)
   - `dataset_v2.json` (97 documentos)

## ⏱️ Tiempo estimado: 18-22 minutos
- Instalación: ~2 min
- Carga modelo: ~2-3 min
- Generación dataset: ~12-15 min (50 preguntas categorizadas con Ground Truth)

---
## 📦 PASO 1: Instalar Dependencias

In [ ]:
%%time
print("="*70)
print("📦 PASO 1: Instalando Dependencias")
print("="*70)
print("\n⏳ Esto tomará ~2-3 minutos...\n")

# Instalar dependencias principales
print("1️⃣ Instalando PyTorch y transformers...")
!pip install -q torch transformers accelerate bitsandbytes

print("\n2️⃣ Instalando sentence-transformers (para embeddings)...")
!pip install -q sentence-transformers

print("\n3️⃣ Instalando FAISS (para búsqueda vectorial)...")
!pip install -q faiss-cpu

print("\n" + "="*70)
print("✅ TODAS LAS DEPENDENCIAS INSTALADAS")
print("="*70)
print("\n📦 Paquetes instalados:")
print("   • torch - PyTorch para GPU")
print("   • transformers - Hugging Face Transformers")
print("   • bitsandbytes - Cuantización 4-bit")
print("   • sentence-transformers - Modelo de embeddings")
print("   • faiss-cpu - Búsqueda vectorial")
print("\n💡 Próximo paso: Verificar GPU")

📦 PASO 1: Instalando Dependencias

⏳ Esto tomará ~2-3 minutos...

1️⃣ Instalando PyTorch y transformers...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.0 MB/s eta 0:00:00

2️⃣ Instalando sentence-transformers (para embeddings)...

3️⃣ Instalando FAISS (para búsqueda vectorial)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.6 MB/s eta 0:00:00

✅ TODAS LAS DEPENDENCIAS INSTALADAS

📦 Paquetes instalados:
   • torch - PyTorch para GPU
   • transformers - Hugging Face Transformers
   • bitsandbytes - Cuantización 4-bit
   • sentence-transformers - Modelo de embeddings
   • faiss-cpu - Búsqueda vectorial

💡 Próximo paso: Verificar GPU
CPU times: user 6.99 s, sys: 960 ms, total: 7.95 s
Wall time: 34.7 s


---
## 🖥️ PASO 2: Verificar GPU

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"✅ GPU Detectada: {gpu_name}")
    print(f"💾 Memoria disponible: {gpu_memory:.2f} GB")
    print("\n⚠️ Nota: Qwen 2.5 14B 4-bit necesita ~8-10 GB de VRAM")
else:
    print("❌ ERROR: No hay GPU disponible")
    print("\n💡 Solución: Ve a Runtime > Change runtime type > Hardware accelerator > GPU")
    raise RuntimeError("GPU no disponible")

✅ GPU Detectada: Tesla T4
💾 Memoria disponible: 14.74 GB

⚠️ Nota: Qwen 2.5 14B 4-bit necesita ~8-10 GB de VRAM


---
## 📤 PASO 3: Subir Archivos

**IMPORTANTE:** Antes de ejecutar esta celda, crea el archivo ZIP en tu PC local:

### PowerShell (Windows):
```powershell
# Navegar al proyecto
cd "E:\Archivos\2025 B\Tesis\Proyecto\Backend_rag"

# Crear carpeta temporal
New-Item -ItemType Directory -Path "temp_colab_simple" -Force

# Copiar solo 2 archivos
Copy-Item "chat_api\rag_service.py" -Destination "temp_colab_simple\"
Copy-Item "data\dataset_v2.json" -Destination "temp_colab_simple\"

# Crear ZIP
Compress-Archive -Path "temp_colab_simple\*" -DestinationPath "backend_colab_simple.zip" -Force

# Limpiar
Remove-Item -Recurse "temp_colab_simple"

Write-Host "✅ backend_colab_simple.zip creado correctamente" -ForegroundColor Green
```

In [ ]:
from google.colab import files
import zipfile
import os

print("📤 Esperando que subas backend_colab_simple.zip...\n")
uploaded = files.upload()

if 'backend_colab_simple.zip' not in uploaded:
    raise FileNotFoundError("❌ No encontré backend_colab_simple.zip")

print("\n📦 Descomprimiendo archivos...")
with zipfile.ZipFile('backend_colab_simple.zip', 'r') as zip_ref:
    zip_ref.extractall('backend_simple')

# Verificar archivos extraídos
required_files = ['rag_service.py', 'dataset_v2.json']
missing = [f for f in required_files if not os.path.exists(f'backend_simple/{f}')]

if missing:
    print(f"\n❌ ERROR: Faltan archivos: {missing}")
    print("\n📋 Archivos encontrados:")
    for root, dirs, files in os.walk('backend_simple'):
        for file in files:
            print(f"  - {os.path.join(root, file)}")
    raise FileNotFoundError(f"Archivos faltantes: {missing}")

print("\n✅ Archivos extraídos correctamente:")
for f in required_files:
    size = os.path.getsize(f'backend_simple/{f}') / 1024
    print(f"  ✓ {f} ({size:.1f} KB)")

📤 Esperando que subas backend_colab_simple.zip...



Saving backend_colab_simple.zip to backend_colab_simple.zip

📦 Descomprimiendo archivos...

✅ Archivos extraídos correctamente:
  ✓ rag_service.py (56.8 KB)
  ✓ dataset_v2.json (72.9 KB)


---
## 🤖 PASO 4: Cargar Modelo Qwen 2.5 14B (4-bit)

**Esto tomará ~2-3 minutos** - El modelo se descarga de Hugging Face (~8 GB)

In [ ]:
%%time
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

print("🤖 Cargando Qwen 2.5 14B Instruct (4-bit)...\n")

# Configuración de cuantización 4-bit (igual que Ollama Q4_K_M)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Cargar modelo
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-14B-Instruct",
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)

# Cargar tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-14B-Instruct",
    trust_remote_code=True
)

print("\n✅ Modelo cargado correctamente")
print(f"💾 Memoria GPU usada: {torch.cuda.memory_allocated(0) / (1024**3):.2f} GB")

🤖 Cargando Qwen 2.5 14B Instruct (4-bit)...



config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


✅ Modelo cargado correctamente
💾 Memoria GPU usada: 9.29 GB
CPU times: user 2min 29s, sys: 1min 59s, total: 4min 29s
Wall time: 11min 3s


---
## 🔧 PASO 5: Importar RAGService

Los archivos ya están parcheados para funcionar sin Django. Solo necesitamos importar y configurar.

---
## 🔗 PASO 4.5: Configurar Rutas de Archivos

RAGService espera los archivos en `/content/data/` pero están en `/content/backend_simple/`

In [ ]:
import os
import shutil

print("🔗 Configurando rutas de archivos...\n")

# Crear carpeta /content/data si no existe
data_dir = '/content/data'
os.makedirs(data_dir, exist_ok=True)
print(f"✅ Carpeta creada: {data_dir}")

# Copiar archivos desde backend_simple a data
source_dir = '/content/backend_simple'
files_to_copy = ['dataset_v2.json', 'rag_service.py']

for filename in files_to_copy:
    source_path = os.path.join(source_dir, filename)
    dest_path = os.path.join(data_dir, filename)

    if os.path.exists(source_path):
        shutil.copy2(source_path, dest_path)
        size = os.path.getsize(dest_path) / 1024
        print(f"✅ Copiado: {filename} ({size:.1f} KB) → {data_dir}")
    else:
        print(f"⚠️ No encontrado: {source_path}")

print("\n🎯 Archivos listos en /content/data/")
print(f"   • dataset_v2.json")
print(f"   • rag_service.py")
print("\n💡 Ahora RAGService podrá encontrar los archivos")

🔗 Configurando rutas de archivos...

✅ Carpeta creada: /content/data
✅ Copiado: dataset_v2.json (72.9 KB) → /content/data
✅ Copiado: rag_service.py (56.8 KB) → /content/data

🎯 Archivos listos en /content/data/
   • dataset_v2.json
   • rag_service.py

💡 Ahora RAGService podrá encontrar los archivos


In [ ]:
import sys
import logging
import os

# Configurar logging para ver qué está pasando
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("="*70)
print("🔧 PASO 5: Importar y configurar RAGService")
print("="*70)

# Verificar que los archivos existen en /content/data
print("\n1️⃣ Verificando archivos en /content/data...")
data_path = '/content/data'
rag_file = os.path.join(data_path, 'rag_service.py')
dataset_file = os.path.join(data_path, 'dataset_v2.json')

if os.path.exists(rag_file):
    print(f"   ✅ rag_service.py encontrado ({os.path.getsize(rag_file)/1024:.1f} KB)")
else:
    print(f"   ❌ No se encuentra: {rag_file}")
    print("\n💡 Solución: Ejecuta el PASO 4.5 primero")
    raise FileNotFoundError(f"❌ No se encuentra: {rag_file}")

if os.path.exists(dataset_file):
    print(f"   ✅ dataset_v2.json encontrado ({os.path.getsize(dataset_file)/1024:.1f} KB)")
else:
    print(f"   ❌ No se encuentra: {dataset_file}")
    print("\n💡 Solución: Ejecuta el PASO 4.5 primero")
    raise FileNotFoundError(f"❌ No se encuentra: {dataset_file}")

# Agregar path para importar desde /content/data
sys.path.insert(0, data_path)

print("\n2️⃣ Importando módulo RAGService...")
print("   ⏳ Esto puede tomar 5-10 segundos...")

try:
    # Importar RAGService
    from rag_service import RAGService
    print("   ✅ Módulo importado correctamente")
except Exception as e:
    print(f"   ❌ Error al importar: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\n3️⃣ Inicializando RAG Service...")
print("   ℹ️  Esto cargará:")
print("      • Modelo de embeddings: paraphrase-multilingual-mpnet-base-v2 (ya en cache)")
print("      • Dataset: 97 documentos desde /content/data/dataset_v2.json")
print("      • Índice FAISS: 97 vectores (768 dimensiones)")
print("   ⏳ Tiempo estimado: 10-20 segundos (modelo ya descargado)...")

try:
    # Crear instancia
    rag_service = RAGService()
    print("\n   ✅ RAG Service inicializado correctamente")
except Exception as e:
    print(f"\n   ❌ Error al inicializar: {e}")
    import traceback
    traceback.print_exc()
    raise

print("\n4️⃣ Verificando componentes...")
print(f"   ✅ Documentos cargados: {len(rag_service.documents)}")

if rag_service.index is not None:
    print(f"   ✅ Índice FAISS: {rag_service.index.ntotal} vectores")
else:
    print(f"   ⚠️ Índice FAISS: None (se creará automáticamente)")

print(f"   ✅ Modelo embeddings: {type(rag_service.model).__name__}")

print("\n" + "="*70)
print("✅ PASO 5 COMPLETADO - RAGService listo para usar")
print("="*70)

🔧 PASO 5: Importar y configurar RAGService

1️⃣ Verificando archivos en /content/data...
   ✅ rag_service.py encontrado (56.8 KB)
   ✅ dataset_v2.json encontrado (72.9 KB)

2️⃣ Importando módulo RAGService...
   ⏳ Esto puede tomar 5-10 segundos...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

   ✅ Módulo importado correctamente

3️⃣ Inicializando RAG Service...
   ℹ️  Esto cargará:
      • Modelo de embeddings: paraphrase-multilingual-mpnet-base-v2 (ya en cache)
      • Dataset: 97 documentos desde /content/data/dataset_v2.json
      • Índice FAISS: 97 vectores (768 dimensiones)
   ⏳ Tiempo estimado: 10-20 segundos (modelo ya descargado)...

   ✅ RAG Service inicializado correctamente

4️⃣ Verificando componentes...
   ✅ Documentos cargados: 97
   ✅ Índice FAISS: 97 vectores
   ✅ Modelo embeddings: SentenceTransformer

✅ PASO 5 COMPLETADO - RAGService listo para usar


---
## 🎯 PASO 6: Inyectar Modelo GPU en RAGService

In [ ]:
print("? Inyectando modelo GPU en RAGService...\n")

# ✅ CRÍTICO: Inyectar modelo y tokenizer GPU en la instancia
rag_service._model = model
rag_service._tokenizer = tokenizer

print("✅ Modelo GPU inyectado correctamente")
print("🎯 RAGService ahora usará GPU automáticamente en lugar de Ollama")

� Inyectando modelo GPU en RAGService...

✅ Modelo GPU inyectado correctamente
🎯 RAGService ahora usará GPU automáticamente en lugar de Ollama


---
## ✅ PASO 7: Test Rápido

In [ ]:
print("🧪 Probando sistema RAG completo (con validación temporal)...\n")

test_question = "¿Cuándo son las fechas de matrícula ordinaria?"
print(f"❓ Pregunta: {test_question}\n")

# ✅ Usar get_answer() que incluye toda la pipeline completa:
# 1. Búsqueda híbrida con pesos adaptativos (5%-95%)
# 2. Construcción de prompt con instrucciones críticas
# 3. Generación con GPU/Ollama
# 4. Validación temporal de fechas (regex)
# 5. Limpieza de respuesta
# 6. Retry automático si detecta alucinaciones
answer = rag_service.get_answer(test_question)

print("\n" + "="*70)
print("💬 RESPUESTA:")
print("="*70)
print(answer)
print("="*70)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧪 Probando sistema RAG completo (con validación temporal)...

❓ Pregunta: ¿Cuándo son las fechas de matrícula ordinaria?


💬 RESPUESTA:
Las fechas de matrícula ordinaria son del **20 al 24 de enero** de 2025.


---
## 🎯 PASO 8: Generar Dataset Completo (50 preguntas con Ground Truth)

**Esto tomará ~12-15 minutos** dependiendo de la GPU

✅ **Características del dataset:**
- **50 preguntas categorizadas** de NotebookLM:
  - 📅 Temporal (15): Fechas límite, vigencias y cronogramas 2025
  - 📋 Factual (15): Costos, contactos y datos específicos
  - 🔗 Integración (10): Procedimientos completos paso a paso
  - 🚫 Restricción (10): Prohibiciones y limitaciones
- **Ground Truth incluido** para evaluación con RAGAS
- **5 contextos** por pregunta (top_k=5)
- **Latencia medida** por pregunta

In [ ]:
%%time
import json
from datetime import datetime
import time
import torch
import gc

# ✅ DATASET COMPLETO: 50 preguntas con Ground Truth de NotebookLM
# Organizado por categorías: Temporal (15) + Factual (15) + Integración (10) + Restricción (10)
test_questions = [
    # === CATEGORÍA: TEMPORAL (15 preguntas) ===
    {"id": 1, "category": "Temporal", "question": "¿Cuál es el rango de fechas exacto para la presentación de expedientes de convalidación a las escuelas en 2025?", "ground_truth": "Del 17 de marzo al 28 de marzo.", "source": "convalidaciones_2025.pdf"},
    {"id": 2, "category": "Temporal", "question": "¿Cuándo es la fecha de inicio de clases para el año académico 2025?", "ground_truth": "31 DE MARZO.", "source": "pautas_matricula_2025a.pdf"},
    {"id": 3, "category": "Temporal", "question": "¿En qué fechas se debe enviar el correo para el Levantamiento de Reserva de Matrícula 2025?", "ground_truth": "Del 20 al 24 de enero de 2025.", "source": "levantamiento_reserva_de_matricula_2025.pdf"},
    {"id": 4, "category": "Temporal", "question": "¿Cuál es el plazo establecido para realizar el trámite de Reactualización de Matrícula 2025?", "ground_truth": "Del 20 de enero al 24 de enero de 2025.", "source": "reactualizacion_2025a.pdf"},
    {"id": 5, "category": "Temporal", "question": "¿Cuál es el periodo para la expedición, registro de Resoluciones y envío a DSA en el proceso de convalidaciones 2025?", "ground_truth": "Del 24 de marzo al 28 de marzo.", "source": "convalidaciones_2025.pdf"},
    {"id": 6, "category": "Temporal", "question": "¿En qué fechas se realiza la modificación de matrículas y matrículas por excepción tras las convalidaciones?", "ground_truth": "Del 28 de marzo al 02 de abril.", "source": "convalidaciones_2025.pdf"},
    {"id": 7, "category": "Temporal", "question": "¿Cuál es la duración mínima de un periodo anual bajo el régimen de estudios de pregrado?", "ground_truth": "Una duración mínima de treintaicuatro (34) semanas.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 8, "category": "Temporal", "question": "¿Por cuánto tiempo máximo consecutivo o alterno puede un estudiante solicitar reserva de matrícula?", "ground_truth": "Por un máximo de tres (03) años consecutivos o alternos.", "source": "pautas_reserva_de_matricula_2025.pdf"},
    {"id": 9, "category": "Temporal", "question": "¿En qué fechas se debe enviar la ficha de matrícula por excepción a la Escuela Profesional en 2025?", "ground_truth": "Del 20 al 24 de enero.", "source": "pautas_matriculaxexcepcion_2025a.pdf"},
    {"id": 10, "category": "Temporal", "question": "¿A qué años aplica la reactualización de matrícula para estudiantes que no registraron matrícula?", "ground_truth": "A estudiantes que no registraron matrícula hasta tres (3) años: el 2022, 2023 o 2024.", "source": "reactualizacion_2025a.pdf"},
    {"id": 11, "category": "Temporal", "question": "¿En qué momento del año académico se presentan las convalidaciones?", "ground_truth": "Solamente se presentan al inicio del año académico Primer Semestre 2025 A.", "source": "convalidaciones_2025.pdf"},
    {"id": 12, "category": "Temporal", "question": "¿Cuál es la duración de un semestre académico según las definiciones del reglamento?", "ground_truth": "Diecisiete (17) semanas.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 13, "category": "Temporal", "question": "Según el Reglamento de Matrícula, ¿a partir de qué año académico es obligatorio acreditar conocimientos de idioma extranjero?", "ground_truth": "A partir del año académico 2021.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 14, "category": "Temporal", "question": "¿Cuándo se procesa el trámite de Reserva de Matrícula?", "ground_truth": "Solamente se tramita en el Primer Semestre.", "source": "pautas_reserva_de_matricula_2025.pdf"},
    {"id": 15, "category": "Temporal", "question": "¿Hasta cuándo procede el retiro de matrícula en un curso por parte del estudiante?", "ground_truth": "Dentro del periodo de matrícula y hasta el consolidado o cierre de matrícula establecido en el cronograma académico aprobado.", "source": "reglamento_matricula_2020_modi.pdf"},

    # === CATEGORÍA: FACTUAL (15 preguntas) ===
    {"id": 16, "category": "Factual", "question": "¿Cuál es el costo del derecho de trámite para el Levantamiento de Reserva de Matrícula 2025?", "ground_truth": "S/ 6.00.", "source": "levantamiento_reserva_de_matricula_2025.pdf"},
    {"id": 17, "category": "Factual", "question": "¿Cuál es el monto a pagar por derechos de reactualización de matrícula 2025?", "ground_truth": "S/ 150.00.", "source": "reactualizacion_2025a.pdf"},
    {"id": 18, "category": "Factual", "question": "¿Cuánto cuesta la tasa por curso de Universidad Particular para la modalidad de ingreso 'Profesionales'?", "ground_truth": "S/ 176.00.", "source": "convalidaciones_2025.pdf"},
    {"id": 19, "category": "Factual", "question": "¿Cuál es la tasa por curso para Traslados Externos provenientes de otra Universidad Nacional?", "ground_truth": "S/ 55.00.", "source": "convalidaciones_2025.pdf"},
    {"id": 20, "category": "Factual", "question": "¿Cuál es el monto exacto por derechos de reserva de matrícula?", "ground_truth": "S/ 50.00.", "source": "pautas_reserva_de_matricula_2025.pdf"},
    {"id": 21, "category": "Factual", "question": "¿Qué artículo del Reglamento de Matrícula define el 'Curso Obligatorio'?", "ground_truth": "Artículo 2.2.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 22, "category": "Factual", "question": "¿Cuál es la dirección física de las Cajas UNSA para realizar pagos presenciales?", "ground_truth": "Calle Moral 316-Cercado.", "source": "convalidaciones_2025.pdf"},
    {"id": 23, "category": "Factual", "question": "¿Cuál es la URL para solicitar el código de pago para modificaciones de matrícula?", "ground_truth": "https://ouis.unsa.edu.pe/unsapay/login.", "source": "pautas_matricula_2025a.pdf"},
    {"id": 24, "category": "Factual", "question": "¿Qué porcentaje mínimo de similitud de contenidos deben tener las asignaturas para ser convalidadas?", "ground_truth": "Por lo menos el 80% de sus contenidos.", "source": "convalidaciones_2025.pdf"},
    {"id": 25, "category": "Factual", "question": "¿Cuál es el número mínimo de créditos en que debe matricularse un estudiante según el Artículo 6 del Reglamento?", "ground_truth": "Un número mínimo de 12 créditos por semestre.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 26, "category": "Factual", "question": "¿Cuántos créditos adicionales pueden matricularse los estudiantes que no tengan ningún curso pendiente de aprobación?", "ground_truth": "Hasta en seis (06) créditos adicionales por semestre.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 27, "category": "Factual", "question": "¿Qué artículo regula la 'Evaluación por jurado'?", "ground_truth": "Artículo 28.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 28, "category": "Factual", "question": "¿Cuál es el correo electrónico oficial para contactar sobre talleres extracurriculares?", "ground_truth": "upacdr_talleres@unsa.edu.pe.", "source": "pautas_matricula_2025a.pdf"},
    {"id": 29, "category": "Factual", "question": "¿En qué banco se debe efectuar el pago del talón de matrícula?", "ground_truth": "Bco. de Crédito del Perú.", "source": "pautas_matricula_2025a.pdf"},
    {"id": 30, "category": "Factual", "question": "¿Cuál es la Ley Universitaria base legal del Reglamento de Matrícula?", "ground_truth": "Ley Nº 30220.", "source": "reglamento_matricula_2020_modi.pdf"},

    # === CATEGORÍA: INTEGRACIÓN (10 preguntas) ===
    {"id": 31, "category": "Integración", "question": "¿Qué documentos exactos se deben enviar en un solo archivo PDF para el Levantamiento de Reserva de Matrícula?", "ground_truth": "1. Solicitud dirigida a dsa@unsa.edu.pe, 2. Recibo de pago por trámite (S/ 6.00), 3. Solicitud enviada al pedir la reserva de matrícula, 4. Copia simple DNI.", "source": "levantamiento_reserva_de_matricula_2025.pdf"},
    {"id": 32, "category": "Integración", "question": "Describa el proceso completo para realizar la Reactualización de Matrícula 2025.", "ground_truth": "1. Pagar los derechos (S/ 150.00) en Caja UNSA o áreas. 2. Enviar la Boleta Electrónica a dsa@unsa.edu.pe indicando el pago. 3. Esperar el registro por la DSA cuando la escuela esté habilitada. 4. Descargar el talón de pago y registrar matrícula en la fecha del Grupo 3.", "source": "reactualizacion_2025a.pdf"},
    {"id": 33, "category": "Integración", "question": "¿Cuáles son los pasos que debe seguir un estudiante para la matrícula por excepción 2025?", "ground_truth": "Descargar la ficha de matrícula de la web, solicitar código de pago en UNSA Pay, pagar en BCP, enviar a la Escuela Profesional la ficha llena y firmada (del 20 al 24 de enero), y esperar evaluación y aprobación del Director y Decano.", "source": "pautas_matriculaxexcepcion_2025a.pdf"},
    {"id": 34, "category": "Integración", "question": "¿Cómo debe conformarse el expediente físico de convalidaciones en cuanto a los documentos requeridos?", "ground_truth": "Solicitud en formato WEB-UNSA, Certificado de estudios original (o apostillado), Sílabo del año aprobado (sellado/firmado), Copia fotostática de constancia de matrícula 2025 A, Recibo de pago total, Copia fotostática de DNI.", "source": "convalidaciones_2025.pdf"},
    {"id": 35, "category": "Integración", "question": "¿Cuál es el procedimiento para rectificar un error u omisión en la matrícula regular?", "ground_truth": "Presentar (enviar) a la Escuela Profesional la Ficha de Matrícula llena y firmada, la constancia de matrícula y el recibo de pago por modificación de matrícula pagado en BCP con código obtenido en UNSA Pay.", "source": "pautas_matricula_2025a.pdf"},
    {"id": 36, "category": "Integración", "question": "¿Qué debe contener el archivo PDF para solicitar la Reserva de Matrícula 2025?", "ground_truth": "Solicitud redactada y firmada indicando años de reserva, Constancia de matrícula del periodo anterior (2024), Recibo de pago por derechos (S/ 50.00), y Copia simple del DNI.", "source": "pautas_reserva_de_matricula_2025.pdf"},
    {"id": 37, "category": "Integración", "question": "¿Cómo debe llenarse el formato de solicitud para convalidaciones si las asignaturas exceden las líneas del formato?", "ground_truth": "Llenará la solicitud en otra(s) hoja(s) (sacar copia de ambos lados) escribiendo 01 asignatura por línea y enumerando todas las hojas en la parte superior.", "source": "convalidaciones_2025.pdf"},
    {"id": 38, "category": "Integración", "question": "¿Cuál es el proceso general de matrícula regular descrito en las pautas 2025 A?", "ground_truth": "Ingresar al portal de matrículas para obtener talón, pagar en BCP, ingresar al Sistema de Registro de matrícula (verifica prerrequisitos y créditos), y finalmente validar matrícula e imprimir Constancia.", "source": "pautas_matricula_2025a.pdf"},
    {"id": 39, "category": "Integración", "question": "¿Qué requisitos deben cumplir los sílabos presentados para convalidación si provienen de otra universidad?", "ground_truth": "Deben ser del año en que aprobó la asignatura, con sellos y firmas del Decano de la Facultad y Secretario General de esa Universidad; si es del extranjero, con la apostilla respectiva.", "source": "convalidaciones_2025.pdf"},
    {"id": 40, "category": "Integración", "question": "¿Qué sucede administrativamente después de que se aprueba una matrícula por excepción?", "ground_truth": "El Director y Decano autorizan firmando la ficha, registran la matrícula en el Sistema habilitado, cierran el Sistema y envían a la DSA la relación de alumnos con el motivo (resumen Sisacad).", "source": "pautas_matriculaxexcepcion_2025a.pdf"},

    # === CATEGORÍA: RESTRICCIÓN (10 preguntas) ===
    {"id": 41, "category": "Restricción", "question": "¿Pueden los ingresantes del año académico 2025 solicitar Reserva de Matrícula?", "ground_truth": "NO pueden solicitar Reserva de Matricula, al no tener el requisito de: matrícula en el año inmediato anterior (2024).", "source": "pautas_reserva_de_matricula_2025.pdf"},
    {"id": 42, "category": "Restricción", "question": "¿Se pueden convalidar cursos aprobados en institutos?", "ground_truth": "No se convalidan cursos aprobados en institutos.", "source": "convalidaciones_2025.pdf"},
    {"id": 43, "category": "Restricción", "question": "¿Es posible realizar la matrícula en horarios superpuestos según el reglamento?", "ground_truth": "Queda prohibida la matrícula en horarios superpuestos.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 44, "category": "Restricción", "question": "¿Qué ocurre si un estudiante deja de matricularse por más de tres años consecutivos o alternos?", "ground_truth": "Deberán volver a postular a la Universidad.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 45, "category": "Restricción", "question": "¿Puede un estudiante separado definitivamente de la Universidad levantar la condición de reserva de matrícula?", "ground_truth": "No puede ser autorizado para levantar la condición de reserva de matrícula, quien haya sido separado definitivamente de la Universidad por razones académicas o disciplinarias.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 46, "category": "Restricción", "question": "¿Se permite validar la matrícula de estudiantes que incumplan el tope máximo de créditos o prerrequisitos?", "ground_truth": "Por ninguna razón se validará la matrícula de estudiantes que incumplan el tope máximo de créditos semestrales establecido por su Escuela Profesional y los prerrequisitos.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 47, "category": "Restricción", "question": "¿Pueden asignarse las vacantes de los ingresantes no matriculados a otros postulantes?", "ground_truth": "Las vacantes de los ingresantes no matriculados, no podrán ser asignadas a ningún otro postulante o persona.", "source": "reglamento_matricula_2020_modi.pdf"},
    {"id": 48, "category": "Restricción", "question": "¿Es posible presentar expedientes de convalidación sin matrícula vigente en la escuela de destino?", "ground_truth": "No, debe tener matricula vigente (2025 A) en la escuela donde presenta expediente.", "source": "convalidaciones_2025.pdf"},
    {"id": 49, "category": "Restricción", "question": "En el expediente de convalidaciones, ¿se pueden omitir cursos prerrequisitos?", "ground_truth": "No, presentar todas las asignaturas que solicite convalidación (sin pasarse cursos prerrequisitos).", "source": "convalidaciones_2025.pdf"},
    {"id": 50, "category": "Restricción", "question": "¿Qué sucede si un estudiante desaprueba una misma materia por cuarta vez?", "ground_truth": "Procede su retiro definitivo.", "source": "reglamento_matricula_2020_modi.pdf"}
]

print(f"🎯 Generando dataset con {len(test_questions)} preguntas categorizadas...\n")
print("   📊 Distribución:")
print("      • Temporal: 15 preguntas")
print("      • Factual: 15 preguntas")
print("      • Integración: 10 preguntas")
print("      • Restricción: 10 preguntas")
print("\n⏱️ Tiempo estimado: ~12-15 minutos (50 preguntas)\n")
print("💾 Memoria GPU inicial: {:.2f} GB usada".format(torch.cuda.memory_allocated(0) / (1024**3)))
print("="*70)
print("="*70)

results = []
latencies = []  # 📊 Lista para medir latencias por pregunta
start_time = datetime.now()

for i, q_item in enumerate(test_questions, 1):
    question_text = q_item["question"]
    print(f"\n[{i}/{len(test_questions)}] 🔍 [{q_item['category']}] {question_text[:50]}...")

    try:
        # ⏱️ MEDIR LATENCIA: Iniciar cronómetro para esta pregunta
        question_start = time.time()

        # ✅ CORREGIDO: Usar get_answer() que incluye TODA la lógica de validación
        # - Búsqueda híbrida con pesos adaptativos
        # - Generación con prompt estructurado
        # - Validación temporal de fechas
        # - Limpieza de respuesta
        # - Retry automático si hay alucinaciones
        answer = rag_service.get_answer(question_text)

        # ⏱️ MEDIR LATENCIA: Detener cronómetro
        question_latency = time.time() - question_start
        latencies.append(question_latency)

        # ✅ MISMO NÚMERO DE CONTEXTOS que dataset local: top_k=5
        relevant_docs = rag_service.search_documents(question_text, top_k=5)
        contexts = [doc['documento']['content'] for doc in relevant_docs]

        # ✅ NUEVO: Almacenar resultado con ground_truth y metadatos
        results.append({
            "id": q_item["id"],
            "category": q_item["category"],
            "question": question_text,
            "answer": answer,
            "ground_truth": q_item["ground_truth"],
            "contexts": contexts,
            "source": q_item["source"],
            "latency_seconds": round(question_latency, 3)
        })

        print(f"    ✅ Completado ({len(answer)} chars) - ⏱️ {question_latency:.2f}s")

        # 🧹 CRÍTICO: Limpiar caché GPU después de cada pregunta
        torch.cuda.empty_cache()
        gc.collect()

        # Mostrar memoria GPU cada 10 preguntas
        if i % 10 == 0:
            mem_used = torch.cuda.memory_allocated(0) / (1024**3)
            print(f"    💾 Memoria GPU: {mem_used:.2f} GB")

    except Exception as e:
        print(f"    ❌ Error: {e}")
        results.append({
            "id": q_item["id"],
            "category": q_item["category"],
            "question": question_text,
            "answer": f"ERROR: {str(e)}",
            "ground_truth": q_item["ground_truth"],
            "contexts": [],
            "source": q_item["source"],
            "latency_seconds": 0.0
        })

        # Limpiar caché incluso si hay error
        torch.cuda.empty_cache()
        gc.collect()

end_time = datetime.now()
elapsed = (end_time - start_time).total_seconds()

print("\n" + "="*70)
print(f"✅ Dataset generado correctamente")
print(f"⏱️ Tiempo total: {elapsed/60:.2f} minutos")
print(f"📊 Preguntas procesadas: {len(results)}/{len(test_questions)}")
print("="*70)

# 📊 ESTADÍSTICAS DE LATENCIA
if latencies:
    avg_latency = sum(latencies) / len(latencies)
    min_latency = min(latencies)
    max_latency = max(latencies)

    print(f"\n⏱️ LATENCIA PROMEDIO: {avg_latency:.2f}s")
    print(f"   Mínima: {min_latency:.2f}s | Máxima: {max_latency:.2f}s")

    # Verificar objetivo < 5.0s
    if avg_latency < 5.0:
        print(f"   ✅ Objetivo cumplido: < 5.0s")
    else:
        print(f"   ⚠️ Por encima del objetivo (5.0s)")

print("="*70)

# ✅ GUARDAR 2 VERSIONES DEL DATASET

# 1️⃣ VERSIÓN RAGAS (solo campos requeridos por RAGAS)
ragas_minimal = []
for item in results:
    if not item['answer'].startswith('ERROR'):  # Excluir errores
        ragas_minimal.append({
            "question": item["question"],
            "answer": item["answer"],
            "ground_truth": item["ground_truth"],
            "contexts": item["contexts"]
        })

ragas_file = 'ragas_evaluation_dataset.json'
with open(ragas_file, 'w', encoding='utf-8') as f:
    json.dump(ragas_minimal, f, ensure_ascii=False, indent=2)

print(f"\n💾 Dataset RAGAS guardado en: {ragas_file}")
print(f"   📌 Solo campos RAGAS: question, answer, ground_truth, contexts")
print(f"   📌 {len(ragas_minimal)} casos válidos (sin errores)")
print(f"   ✅ Listo para evaluate() directo")

# 2️⃣ VERSIÓN COMPLETA (con todos los metadatos)
full_file = 'ragas_evaluation_dataset_full.json'
with open(full_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n💾 Dataset COMPLETO guardado en: {full_file}")
print(f"   📌 Incluye: id, category, source, latency_seconds")
print(f"   📌 {len(results)} casos totales (incluye errores si los hay)")
print(f"   ✅ Útil para análisis detallado")

🎯 Generando dataset con 50 preguntas categorizadas...

   📊 Distribución:
      • Temporal: 15 preguntas
      • Factual: 15 preguntas
      • Integración: 10 preguntas
      • Restricción: 10 preguntas

⏱️ Tiempo estimado: ~12-15 minutos (50 preguntas)

💾 Memoria GPU inicial: 11.37 GB usada

[1/50] 🔍 [Temporal] ¿Cuál es el rango de fechas exacto para la present...
    ✅ Completado (38 chars) - ⏱️ 5.49s

[2/50] 🔍 [Temporal] ¿Cuándo es la fecha de inicio de clases para el añ...
    ✅ Completado (78 chars) - ⏱️ 6.65s

[3/50] 🔍 [Temporal] ¿En qué fechas se debe enviar el correo para el Le...
    ✅ Completado (139 chars) - ⏱️ 9.90s

[4/50] 🔍 [Temporal] ¿Cuál es el plazo establecido para realizar el trá...
    ✅ Completado (133 chars) - ⏱️ 10.57s

[5/50] 🔍 [Temporal] ¿Cuál es el periodo para la expedición, registro d...
    ✅ Completado (150 chars) - ⏱️ 9.96s

[6/50] 🔍 [Temporal] ¿En qué fechas se realiza la modificación de matrí...
    ✅ Completado (132 chars) - ⏱️ 8.32s

[7/50] 🔍 [Tempora

---
## 📊 PASO 9: Estadísticas del Dataset

In [ ]:
print("📊 ESTADÍSTICAS DEL DATASET\n")
print("="*70)

total = len(results)
successful = sum(1 for r in results if not r['answer'].startswith('ERROR'))
errors = total - successful

print(f"Total de casos: {total}")
print(f"✅ Exitosos: {successful} ({successful/total*100:.1f}%)")
print(f"❌ Errores: {errors} ({errors/total*100:.1f}%)")

# ✅ NUEVO: Estadísticas por categoría
from collections import Counter
category_counts = Counter(r['category'] for r in results if not r['answer'].startswith('ERROR'))
print(f"\n📂 Distribución por categoría (exitosos):")
for cat, count in sorted(category_counts.items()):
    print(f"  {cat}: {count} preguntas")

# Estadísticas de longitud
answer_lengths = [len(r['answer']) for r in results if not r['answer'].startswith('ERROR')]
if answer_lengths:
    print(f"\n📏 Longitud de respuestas:")
    print(f"  Promedio: {sum(answer_lengths)/len(answer_lengths):.0f} caracteres")
    print(f"  Mínima: {min(answer_lengths)} caracteres")
    print(f"  Máxima: {max(answer_lengths)} caracteres")

# Promedio de contextos
contexts_per_q = [len(r['contexts']) for r in results]
print(f"\n📚 Contextos por pregunta:")
print(f"  Promedio: {sum(contexts_per_q)/len(contexts_per_q):.1f} documentos")

# ⏱️ ESTADÍSTICAS DE LATENCIA (solo casos exitosos)
successful_latencies = [r['latency_seconds'] for r in results if not r['answer'].startswith('ERROR') and r['latency_seconds'] > 0]
if successful_latencies:
    avg_lat = sum(successful_latencies) / len(successful_latencies)
    print(f"\n⏱️ Latencia (solo casos exitosos):")
    print(f"  Promedio: {avg_lat:.3f}s")
    print(f"  Mínima: {min(successful_latencies):.3f}s")
    print(f"  Máxima: {max(successful_latencies):.3f}s")

    # Evaluación vs objetivo
    emoji = "✅" if avg_lat < 5.0 else "⚠️"
    print(f"  {emoji} Objetivo < 5.0s: {'CUMPLIDO' if avg_lat < 5.0 else 'NO CUMPLIDO'}")

print("\n" + "="*70)

# Mostrar ejemplo
if successful > 0:
    ejemplo = next(r for r in results if not r['answer'].startswith('ERROR'))
    print("\n📄 EJEMPLO DE CASO GENERADO:\n")
    print(f"ID: {ejemplo['id']}")
    print(f"Categoría: {ejemplo['category']}")
    print(f"Pregunta: {ejemplo['question']}")
    print(f"\nRespuesta: {ejemplo['answer'][:150]}...")
    print(f"\nGround Truth: {ejemplo['ground_truth']}")
    print(f"\nContextos: {len(ejemplo['contexts'])} documentos")
    print(f"Fuente: {ejemplo['source']}")
    print(f"Latencia: {ejemplo['latency_seconds']:.3f}s")

📊 ESTADÍSTICAS DEL DATASET

Total de casos: 50
✅ Exitosos: 50 (100.0%)
❌ Errores: 0 (0.0%)

📂 Distribución por categoría (exitosos):
  Factual: 15 preguntas
  Integración: 10 preguntas
  Restricción: 10 preguntas
  Temporal: 15 preguntas

📏 Longitud de respuestas:
  Promedio: 258 caracteres
  Mínima: 38 caracteres
  Máxima: 1290 caracteres

📚 Contextos por pregunta:
  Promedio: 5.0 documentos

⏱️ Latencia (solo casos exitosos):
  Promedio: 13.487s
  Mínima: 4.747s
  Máxima: 58.720s
  ⚠️ Objetivo < 5.0s: NO CUMPLIDO


📄 EJEMPLO DE CASO GENERADO:

ID: 1
Categoría: Temporal
Pregunta: ¿Cuál es el rango de fechas exacto para la presentación de expedientes de convalidación a las escuelas en 2025?

Respuesta: Del **17 de marzo** al **28 de marzo**...

Ground Truth: Del 17 de marzo al 28 de marzo.

Contextos: 5 documentos
Fuente: convalidaciones_2025.pdf
Latencia: 5.486s


---
## 📥 PASO 10: Descargar Resultados

Ejecuta las celdas según el archivo que necesites descargar:
- **Celda 10.1**: Dataset RAGAS (solo campos para evaluación)
- **Celda 10.2**: Dataset completo (con todos los metadatos)

### 📥 10.1 - Descargar Dataset RAGAS (Formato Puro)

Archivo: `ragas_evaluation_dataset.json`
- ✅ Solo 4 campos: question, answer, ground_truth, contexts
- ✅ Sin errores (solo casos exitosos)
- ✅ Listo para `evaluate()` de RAGAS

In [ ]:
from google.colab import files

print("📥 Descargando ragas_evaluation_dataset.json...")
print("   📌 Formato: Solo campos RAGAS")
print("   📌 Uso: Evaluación directa con RAGAS\n")

files.download('ragas_evaluation_dataset.json')

print("\n✅ Descarga iniciada")
print("💡 Guarda en: Backend_rag/evaluation/ragas_evaluation_dataset.json")

📥 Descargando ragas_evaluation_dataset.json...
   📌 Formato: Solo campos RAGAS
   📌 Uso: Evaluación directa con RAGAS



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Descarga iniciada
💡 Guarda en: Backend_rag/evaluation/ragas_evaluation_dataset.json


### 📥 10.2 - Descargar Dataset Completo (Con Metadatos)

Archivo: `ragas_evaluation_dataset_full.json`
- ✅ Incluye: id, category, source, latency_seconds
- ✅ Incluye errores (si los hay)
- ✅ Útil para análisis detallado por categoría

In [ ]:
from google.colab import files

print("📥 Descargando ragas_evaluation_dataset_full.json...")
print("   📌 Formato: Completo con metadatos")
print("   📌 Uso: Análisis detallado, debugging, estadísticas\n")

files.download('ragas_evaluation_dataset_full.json')

print("\n✅ Descarga iniciada")
print("💡 Guarda en: Backend_rag/evaluation/ragas_evaluation_dataset_full.json")

📥 Descargando ragas_evaluation_dataset_full.json...
   📌 Formato: Completo con metadatos
   📌 Uso: Análisis detallado, debugging, estadísticas



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Descarga iniciada
💡 Guarda en: Backend_rag/evaluation/ragas_evaluation_dataset_full.json


---
## ✅ ¡LISTO!

### 🎉 Has generado exitosamente 2 datasets RAGAS

**Archivos generados:**
1. **`ragas_evaluation_dataset.json`** - Formato puro RAGAS (4 campos: question, answer, ground_truth, contexts)
2. **`ragas_evaluation_dataset_full.json`** - Versión completa con metadatos (id, category, source, latency_seconds)

**Próximos pasos:**
1. Descarga ambos archivos
2. Guárdalos en tu proyecto local: `Backend_rag/evaluation/`
3. Usa el archivo RAGAS para evaluar: `evaluate_with_ragas.ipynb`
4. Usa el archivo completo para análisis detallado por categoría

### 📊 Comparación con Local:
- **Local (CPU):** ~45-60 minutos (50 preguntas)
- **Colab (GPU):** ~12-15 minutos (50 preguntas)
- **Speedup:** 3-4x más rápido ⚡

### 🎯 Dataset Mejorado:
- ✅ **50 preguntas categorizadas** (vs 15 anteriores)
- ✅ **Ground Truth incluido** de NotebookLM
- ✅ **4 categorías:** Temporal, Factual, Integración, Restricción
- ✅ **Compatible con RAGAS** para evaluación completa

### 💰 Costo:
- **Google Colab:** GRATIS (con GPU T4)
- **Ollama Local:** Gratis pero más lento

---

**¿Preguntas o problemas?** Revisa los logs de cada celda para debugging.